![](https://drive.google.com/uc?export=view&id=1-5X9OUkA-C2Ih1gOS9Jd7GmkTWUEpDg1)

**Asignatura:** *Programación Científica*

**Profesores:**

*   Dr. Juan Bekios Calfa

**Ayudantes**:

* Martín Puebla Rivera (Laboratorio) martin.puebla@alumnos.ucn.cl
* Pablo Cortés Vega (Cátedra) pablo.cortes01@alumnos.ucn.cl
    

**Fecha:** 16 de mayo de 2025

---

# Laboratorio 2: Funciones y carga de datos con Pandas

## Descripción del problema:

El estudio del flujo de peatones en un corredor unidireccional es un problema relevante en la planificación urbana y en la gestión de eventos donde grandes grupos de personas transitan por espacios limitados.

Este problema se centra en analizar y entender cómo se mueven los individuos en un espacio lineal y restringido, donde todos se desplazan en la misma dirección.

Se busca identificar patrones, calcular velocidades promedio y evaluar cómo la densidad y otros factores afectan la movilidad de los peatones.


## Objetivos:

1. Cargar el dataset utilizando Pandas.
2. Implementar el problema utilizando funciones.
3. Calcular la cantidad de peatones que participan en el experimento a partir del dataset.
4. Calcular la velocidad media de cada peatón.
5. Graficar un histograma de todas las velocidades medias y establecer cuál es la velocidad media dominante.
6. Calcular la velocidad media de todos los peatones en el experimento.
7. Calcular la distancia de los k peatones más cercanos por frame para cada peatón del frame.

### Dataset de peatones:

Información general del problema y dataset: https://ped.fz-juelich.de/da/doku.php?id=corridor5

Archivo de texto a procesar:

* [UNI_CORR_500_05](https://ped.fz-juelich.de/experiments/2013.06.19_Duesseldorf_Messe_BaSiGo/result/corrected/UNI_CORR_500_Gregor/traj_UNI_CORR_500_05.txt)
* [UNI_CORR_500_09](http://ped.fz-juelich.de/experiments/2013.06.19_Duesseldorf_Messe_BaSiGo/result/corrected/UNI_CORR_500_Gregor/traj_UNI_CORR_500_09.txt)


### Procedimiento general del laboratorio:

*  **Datos:** Los datos consistirán en registros temporales de peatones que transitan por un corredor. Cada registro incluirá marcas de tiempo, posiciones y otros posibles atributos como la edad o el género del peatón.
*  **Herramientas de software:** Se utilizará Python como lenguaje de programación principal. El laboratorio utilizará los conceptos de manipulación de strings y diccionarios para la implementación del problema.
*  **Métodos de procesamiento de datos:** Se emplearán técnicas de manipulación de cadenas para limpiar y preparar los datos, y estructuras de datos como listas y diccionarios para su organización y análisis.
*  **Entregables:**  El laboratorio finalizará con la presentación de un archivo colab que incluya gráficos y tablas que resuman los hallazgos estadísticos, así como una discusión sobre las implicaciones de estos resultados para el diseño de infraestructura peatonal.

Este laboratorio busca dotar a los estudiantes o investigadores de las herramientas y métodos necesarios para abordar problemas de ingeniería urbana y de transporte de manera práctica y basada en datos.

# 1. Ejemplo de carga de datos

Estos ejemplos están diseñados para ilustrar los componentes esenciales involucrados en la carga, análisis y visualización de datos, culminando con el cálculo de la velocidad media.

Aunque el ejemplo proporcionado se centra en la carga de datos de una sola persona, se espera que los estudiantes apliquen los conceptos aprendidos para abordar el problema completo tal como se ha planteado.

##1.1 Carga de datos

Se cargarán los datos y se acondicionarán para poder utilizarlos adecuadamente en las fase de análisis.

### 1.1.1 Se monta el disco virtual de google drive.

In [ ]:
# Montar unidad virtual desde google Drive
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
%cd '/gdrive/MyDrive/PCIENTIFICA/lab04/dataset'

### 1.1.2 Carga de datos utilizando Pandas

In [ ]:
import pandas as pd

# Cargamos el archivo UNI_CORR_500_05
data = pd.read_csv('traj_UNI_CORR_500_05.txt')

# Mostrar las primeras filas del dataset para entender su estructura
data.head()

### 1.1.3 Carga correcta utilizando Pandas

Para poder cargar correctamente el archivo de texto de peatones debemos considerar que las dos primeras líneas tienen información de cabecera que es irrelevante y no se debe considerar. También se debe indicar que los separadores de texto no son ni comas, o puntos y comas. En este caso son tabuladores.




In [ ]:
# Cargar el dataset omitiendo las primeras dos líneas y especificando el separador de tabulación
data = pd.read_csv('traj_UNI_CORR_500_05.txt', skiprows=2, sep='\t')

# Mostrar las primeras filas del dataset para confirmar que se ha cargado correctamente
data.head()

### 1.1.4 Cambiar el nombre de una columna

Para mayor claridad la columna `# PersID` será cambiada por `ID`.








In [ ]:
# Renombrar la columna '# PersID' a 'ID'
data.rename(columns={'# PersID': 'ID'}, inplace=True)

# Mostrar las primeras filas del dataset para confirmar que se ha cargado y renombrado correctamente
data.head()

## 1.2 Ejercicios

### 1.2.1 Calcular la cantidad de peatones que participan en el experimento a partir del dataset.

In [ ]:
print(f'Cantidad de peatones en el dataset: {data["ID"].nunique()}')

###1.2.2 Calcular la velocidad media de cada peatón

Para calcular la velocidad de cada peatón primero se debe calcular la distancia recorrida en cada frame por cada peatón.

Para su implementación se realizarán los siguientes pasos:

1. Se define una función calcular_distancia que calcula la distancia euclidiana entre cada fila y la anterior dentro de un grupo.

2. Se utiliza `groupby` para aplicar esta función a cada grupo de ID con el método `apply`.

3. Se muestran las distancias calculadas dentro del DataFrame.


In [ ]:
%%time
import numpy as np

# Función para calcular la distancia euclidiana
def calcular_distancia(df):
    df['distancia'] = np.sqrt((df['X'] - df['X'].shift(1))**2 + (df['Y'] - df['Y'].shift(1))**2)
    df['distancia'].fillna(0, inplace=True)  # Llenar los valores NaN con 0
    return df

# Aplicar la función de distancia por cada grupo de ID
data = data.groupby('ID').apply(calcular_distancia)

# Restablecer el índice para "desagrupar"
data = data.reset_index(drop=True)

# Mostrar el DataFrame con las distancias calculadas
print(data)

In [ ]:
data.head()

In [ ]:
# Calcular las velocidades por peatón
def calcular_velocidad(df):
    df['velocidad'] = df['distancia']/0.04 #(1/25 seg)
    return df

# Aplicar la función de distancia por cada grupo de ID
data = data.groupby('ID').apply(calcular_velocidad)

# Restablecer el índice para "desagrupar"
data = data.reset_index(drop=True)

# Mostrar el DataFrame con las distancias calculadas
print(data)

###1.2.3 Eliminar todas las velocidades cero

In [ ]:
# Eliminar las filas donde la distancia es cero
data = data[data['distancia'] != 0]

In [ ]:
data.head()

Utilizando el comando drop

In [ ]:
# Identificar las filas donde la distancia es cero
indice_eliminar_filas = data[data['distancia'] == 0].index

# Eliminar las filas identificadas
data = data.drop(indice_eliminar_filas)

In [ ]:
data.head()

###1.2.3 Calcular la velocidad media de una persona

In [ ]:
dp = data[data['ID'] == 1]

print(f'La velocidad media de la persona 1 es: {dp["velocidad"].mean()}')

###1.2.4 Calcular la velocidad media de todos los peatones

In [ ]:
df_velocidad_media = data.groupby('ID')['velocidad'].mean()
df_velocidad_media.head()

#2. Problema

El laboratorio busca analizar el movimiento de peatones en un corredor unidireccional para comprender mejor la dinámica de flujo peatonal en entornos controlados. Utilizando un conjunto de datos específico, el laboratorio se enfoca en:

1. **Cargar y Preparar los Datos:** Importar los datos desde un archivo externo utilizando Python, con especial atención en extraer información relevante como marcas de tiempo y posiciones de cada peatón registrado.

2. **Análisis de Datos:**
   - **Cantidad de Peatones:** Determinar cuántos peatones participan en el experimento, contando el número de identificadores únicos de peatones en el dataset.
   - **Velocidad Media por Peatón:** Calcular la velocidad media para cada peatón individualmente. Esto implicará determinar la distancia total recorrida y el tiempo total empleado, luego calcular la velocidad promedio para cada uno.
   
3. **Visualización de Datos:**
   - **Histograma de Velocidades Medias:** Crear un histograma para visualizar la distribución de las velocidades medias de todos los peatones. Identificar la velocidad media dominante observando el peak (valor más alto) más prominente en el histograma.
   
4. **Cálculo de Velocidad Media Global:** Determinar la velocidad media de todos los peatones en el experimento, que será el promedio de todas las velocidades medias individuales.

5. **Calcular la distancia de los k peatones más cercanos** por frame para cada peatón del frame.


### Objetivos Adicionales:

- **Descomposición funcional del problema**: Asegurar que cada parte del análisis se maneje a través de funciones específicas y modulares. Esto incluye cargar datos, calcular distancias, velocidades y realizar análisis específicos, todo encapsulado en funciones para facilitar la comprensión y reutilización del código.

- **Documentar el código utilizando docstring**: Revisar el siguiente link, https://www.geeksforgeeks.org/python-docstrings/.

- **Interpretación de Resultados**: Discutir cómo los resultados pueden influir en el diseño de infraestructura peatonal y en la planificación de eventos masivos, basándose en las velocidades medias y las tendencias observadas en el histograma.

Este enfoque permitirá obtener una comprensión detallada de las características del flujo peatonal, crucial para mejoras en seguridad y eficiencia en entornos urbanos y otros espacios públicos.

## 1. Cargar y Preparar los Datos

### 1.1 Carga de archivos de textos de peatones

Declarar solo las funciones.

In [ ]:
# Función para cargar el dataset
def cargar_datos(ruta):
    """
    Carga el dataset desde la ruta especificada.

    Args:
    ruta (str): Ruta al archivo CSV del dataset.

    Returns:
    DataFrame: DataFrame de Pandas con los datos cargados.
    """
    # Agregar código aqui

# Cargar el dataset
data = cargar_datos('ruta/del/dataset.csv')

##2. Análisis de Datos

**Cantidad de Peatones:** Determinar cuántos peatones participan en el experimento, contando el número de identificadores únicos de peatones en el dataset.

**Velocidad Media por Peatón:** Calcular la velocidad media para cada peatón individualmente. Esto implicará determinar la distancia total recorrida y el tiempo total empleado, luego calcular la velocidad promedio para cada uno.


In [ ]:
# Calcular la cantidad de peatones que hay en el dataset
def contar_peatones(data):
    """
    Calcula la cantidad de peatones únicos en el dataset.

    Args:
    data (DataFrame): DataFrame con los datos de los peatones.

    Returns:
    int: Cantidad de peatones únicos.
    """
    # Escribir código aquí

# Calcular la cantidad de peatones únicos
num_peatones = contar_peatones(data)
print(f'Cantidad de peatones: {num_peatones}')

In [ ]:
# Función para calcular la velocidad media de cada peatón
def calcular_velocidades_medias(data):
    """
    Calcula la velocidad media de cada peatón en el dataset.

    Args:
    data (DataFrame): DataFrame con los datos de los peatones.

    Returns:
    DataFrame: DataFrame con las velocidades medias de cada peatón.
    """
    # Escribir código aquí

# Calcular las velocidades medias
velocidades_df = calcular_velocidades_medias(data)

# Mostrar las primeras filas del DataFrame de velocidades
velocidades_df.head()

##3. Visualización de Datos

 Crear un histograma para visualizar la distribución de las velocidades medias de todos los peatones. Identificar la velocidad media dominante observando el peak (valor más alto) más prominente en el histograma.



In [ ]:
import matplotlib.pyplot as plt

# Función para graficar el histograma de las velocidades medias
def graficar_histograma(velocidades_df):
    """
    Grafica un histograma de las velocidades medias de los peatones.

    Args:
    velocidades_df (DataFrame): DataFrame con las velocidades medias de los peatones.
    """
    plt.hist(velocidades_df['velocidad_media'], bins=10, edgecolor='black')
    plt.xlabel('Velocidad Media (m/s)')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de Velocidades Medias de Peatones')
    plt.show()

# Graficar el histograma de las velocidades medias

# Escribir código para obtener velocidades_df aquí
graficar_histograma(velocidades_df)

# Calcular la velocidad media dominante
velocidad_media_dominante = velocidades_df['velocidad_media'].mode()[0]
print(f'Velocidad media dominante: {velocidad_media_dominante} m/s')

##4. Cálculo de Velocidad Media Global

Determinar la velocidad media de todos los peatones en el experimento, que será el promedio de todas las velocidades medias individuales.

In [ ]:
# Función para calcular la velocidad media total de todos los peatones
def calcular_velocidad_media_total(velocidades_df):
    """
    Calcula la velocidad media total de todos los peatones en el dataset.

    Args:
    velocidades_df (DataFrame): DataFrame con las velocidades medias de los peatones.

    Returns:
    float: Velocidad media total de todos los peatones.
    """
    # Agregar código aquí

# Calcular la velocidad media total de todos los peatones
velocidad_media_total = calcular_velocidad_media_total(velocidades_df)
print(f'Velocidad media total de todos los peatones: {velocidad_media_total} m/s')

##5. Calcular los k vecinos más cercanos de un peatón en un frame

Definir la función que permita obtener los k vecinos más cercanos de todos los peatones en todos los frames. Retornar una lista de tuplas (id peatón, distancia).

In [ ]:
# Definir la función aquí

## 6. Utilizar todas las funciones implementadas para probar los dos archivos

In [ ]:
# Programa principal
def main(archivo, k=3):
  """
  Programa principipal donde se ejecutan todas las tareas solicitadas.

  Args:
  archivo (String): Path o ruta y nombre del archivo.

  Returns:
  float: Velocidad media total de todos los peatones.
  """
  # Agregar código aquí
  return cantidad_personas, velocidad_media_cada_peaton, velocidad_media_total, calcular_vecinos_cercanos_frames

In [ ]:
# Ejecución main()
nombre_archivo = 'archivo.txt'
cantidad_personas, velocidad_media_cada_peaton, velocidad_media_total, calcular_vecinos_cercanos_frames = main(archivo=nombre_archivo, k=3)

# Graficar los resultados obtenidos por medio de histograma u otro gráfico

# Instrucciones para completar el informe del laboratorio: Análisis del Flujo Peatonal

## 1. Sección de Resultados

En esta sección debes **mostrar y explicar los resultados obtenidos** tras procesar los datos del experimento. Incluye tanto **tablas como gráficos** que respalden tus hallazgos. No basta con solo mostrar números: es importante que expliques qué significan esos datos en el contexto del flujo peatonal.

Debes considerar incluir (y explicar) métricas como:

- Cantidad total de peatones que aparecen en el experimento.
- Velocidad media **por peatón** y **general**.
- Histograma de velocidades para identificar la velocidad más frecuente.
- Distancia media a los `k` vecinos más cercanos (por frame).

### Ejemplo de tabla de resultados

> ⚠️ *Este es solo un ejemplo referencial. Debes completarlo con los datos obtenidos por tu grupo.*

| **Métrica**                                                                 | **Valor**                       |
|------------------------------------------------------------------------------|----------------------------------|
| Cantidad total de peatones                                                  | 78                               |
| Velocidad media de cada peatón (promedio por individuo)                     | Ver histograma y lista anexa     |
| Velocidad media general del experimento                                     | 1.49 m/s                         |
| Velocidad más frecuente (moda del histograma de velocidades)                | 1.41 m/s                         |
| Velocidad mínima registrada                                                 | 0.62 m/s                         |
| Velocidad máxima registrada                                                 | 2.18 m/s                         |
| Número de vecinos más cercanos considerados (`k`)                           | 3                                |
| Distancia media a los `k` peatones más cercanos por frame                   | 1.25 m                           |
| Distancia media de los `k` vecinos más cercanos para cada peatón por frame  | Ver distribución por individuo   |

También se deben incluir visualizaciones como:

- Histograma de velocidades.
- Gráfico de dispersión de distancias promedio por peatón.
- Evolución temporal de velocidad general (si es posible).

---

## 2. Sección de Conclusiones

En esta parte debes **reflexionar sobre los resultados obtenidos** y qué implicancias tienen. Puedes guiarte por las siguientes preguntas para redactar tus conclusiones:

- ¿Qué patrones o comportamientos relevantes se observaron en el flujo peatonal?
- ¿Cómo afecta la densidad de peatones a la velocidad observada?
- ¿Qué tan útil fue usar Python para este tipo de análisis?
- ¿Qué limitaciones tuvo el análisis realizado?
- ¿Cómo podrían aplicarse estos resultados en el diseño de espacios públicos o planificación urbana?
- ¿Qué aprendiste sobre el manejo y procesamiento de archivos de texto con Python?

La idea es que esta sección muestre que comprendiste el propósito del análisis, cómo usar herramientas computacionales para estudiar fenómenos reales, y que seas capaz de **extraer aprendizajes útiles y aplicables** a otras situaciones.

---

✍️ **Recuerda incluir los nombres de todos los integrantes del grupo en el notebook.**  
📁 El archivo debe subirse con el nombre correspondiente, por ejemplo: `GRUPO_A.ipynb`



